In [1]:
import sys
sys.path.append('../')

import seaborn as sns
from data_model import Individual
from sys_utils import load_model

checkpoint_path = '../checkpoints_dev'

individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )

from tqdm import tqdm

all_page = []
for ind in tqdm(individuals):
    try:
        for page in ind.wikipedia_pages:
            res = page.dict()
            res['wikidata_id'] = ind.id.wikidata_id
            all_page.append(res)
    except:
        pass
    
template = 'simple_white'

import pandas as pd

df_proxies_all = pd.DataFrame(all_page)
df_proxies = df_proxies_all[~df_proxies_all['language'].str.contains('commons.wikimedia')]
df_proxies = df_proxies.groupby('wikidata_id').sum()

100%|████████████████████████████████| 225636/225636 [00:11<00:00, 19130.45it/s]


In [3]:
# cultural index and count for Low Countries

In [20]:
individuals_filtered[1].

Individual(id=RawIndividual(wikidata_id='Q6121397', name='Santos Domínguez y Benguria', birthyear=1841, gender=['male'], raw_nationalities=[RawNationality(wikidata_id='Q29', name='Spain', location='Point(-3.5 40.2)')], raw_birthcities=None, occupations=[Occupation(wikidata_id='Q42973', name='architect', category=['artist'])]), impact_years=(1850, 1900), cultural_score=0.10024749047065873, country=Country(name='Spain', code='es'), wikipedia_pages=[WikipediaPage(url='https://es.wikipedia.org/wiki/Santos_Dom%C3%ADnguez_y_Benguria', language='es', links_ext_count=5, links_out_count=25, links_in_count=9, author='Doctor seisdedos', author_editcount=2986, editors=16, minor_edits=9, revisions=26, pageviews=23, characters=1187, references=0, unique_references=0, words=199, created_at='2006-11-19')], regions=['re_western_europe', 're_spain'], identifiers=[ExternalID(wikidata_id='P213', name='ISNI'), ExternalID(wikidata_id='P214', name='VIAF ID'), ExternalID(wikidata_id='P244', name='Library of C

In [21]:
individuals_filtered = [x for x in individuals if x.regions != None]

df_individuals = [
    {
        "wikidata_id": x.id.wikidata_id,
        "region_code": x.regions,
        "cultural_score":x.cultural_score,
         "year": x.impact_years,
        "birthyear":x.id.birthyear
    }
    for x in individuals_filtered
]


from data_model_region import Region
df = pd.DataFrame(df_individuals)
df = df.explode('region_code')

regions = load_model(
    Region, name=checkpoint_path + "/regions.jsonl"
)

df_regions = [
        {
            "region_code": x.code,
            "region_name": x.name,
        }
        for x in regions
    ]

df_regions = pd.DataFrame(df_regions)
df = pd.merge(df, df_regions, on = 'region_code')


In [22]:
region = 'Low countries'

In [27]:
df_filtered = df[df['region_name']==region]
df_filtered = df_filtered.dropna()
df_filtered['decade'] = df_filtered['birthyear'].apply(lambda x: round(x / 10) * 10)

In [34]:
df_final = pd.merge(df_proxies, df_filtered, on = 'wikidata_id')

In [76]:
df_count = df_final.groupby(['decade'])['wikidata_id'].count().reset_index()
df_count.columns = ['decade', 'count_individuals']
df_others = df_final.groupby(['decade']).sum().reset_index()
df_fig = pd.merge(df_others, df_count, on = 'decade')


from sklearn.preprocessing import MinMaxScaler
# create a MinMaxScaler object
scaler = MinMaxScaler()

# fit and transform the dataframe
df_normalized = pd.DataFrame(scaler.fit_transform(df_fig), columns=df_fig.columns)
df_normalized['decade'] = df_fig['decade']
df_normalized = df_normalized.drop('birthyear', 1)
df_normalized = df_normalized.set_index('decade').unstack().reset_index()
df_normalized.columns = ['proxy', 'decade','value']

df_normalized = df_normalized[(df_normalized['decade']>=1200)&(df_normalized['decade']<=1850)]

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_26265/1638368517.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_normalized = df_normalized.drop('birthyear', 1)


In [81]:
import plotly.express as px
fig = px.line(df_normalized, 
              x = 'decade', 
              y = 'value', 
              color = 'proxy', 
              width = 1000, 
              height = 600, 
              title = "Low Countries - Correlation between Proxies", 
              template = 'plotly_white')

fig.update_xaxes(showgrid=False, showticklabels=True, zeroline=False)
fig.update_yaxes(showgrid=False, showticklabels=True, zeroline=False)
fig.update_layout(title_font=dict(size=20))

fig.update_layout(
    xaxis_title="",
    yaxis_title='Proxies')
import plotly
plotly.offline.plot(fig)
#fig.write_image(f"save_figures/identifiers_line.png", scale=7)

'temp-plot.html'